<a href="https://colab.research.google.com/github/ambicapvs/ambica_info5731_spring2021/blob/main/In_class_exercise_05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## The fifth In-class-exercise (2/23/2021, 20 points in total)

In exercise-03, I asked you to collected 500 textual data based on your own information needs (If you didn't collect the textual data, you should recollect for this exercise). Now we need to think about how to represent the textual data for text classification. In this exercise, you are required to select 10 types of features (10 types of features but absolutely more than 10 features) in the followings feature list, then represent the 500 texts with these features. The output should be in the following format:
![image.png](attachment:image.png)

The feature list:

* (1) tf-idf features
* (2) POS-tag features: number of adjective, adverb, auxiliary, punctuation, complementizer, coordinating conjunction, subordinating conjunction, determiner, interjection, noun, possessor, preposition, pronoun, quantifier, verb, and other. (select some of them if you use pos-tag features)
* (3) Linguistic features:
  * number of right-branching nodes across all constituent types
  * number of right-branching nodes for NPs only
  * number of left-branching nodes across all constituent types
  * number of left-branching nodes for NPs only
  * number of premodifiers across all constituent types
  * number of premodifiers within NPs only
  * number of postmodifiers across all constituent types
  * number of postmodifiers within NPs only
  * branching index across all constituent types, i.e. the number of right-branching nodes minus number of left-branching nodes
  * branching index for NPs only
  * branching weight index: number of tokens covered by right-branching nodes minus number of tokens covered by left-branching nodes across all categories
  * branching weight index for NPs only 
  * modification index, i.e. the number of premodifiers minus the number of postmodifiers across all categories
  * modification index for NPs only
  * modification weight index: length in tokens of all premodifiers minus length in tokens of all postmodifiers across all categories
  * modification weight index for NPs only
  * coordination balance, i.e. the maximal length difference in coordinated constituents
  
  * density (density can be calculated using the ratio of folowing function words to content words) of determiners/quantifiers
  * density of pronouns
  * density of prepositions
  * density of punctuation marks, specifically commas and semicolons
  * density of auxiliary verbs
  * density of conjunctions
  * density of different pronoun types: Wh, 1st, 2nd, and 3rd person pronouns
  
  * maximal and average NP length
  * maximal and average AJP length
  * maximal and average PP length
  * maximal and average AVP length
  * sentence length

* Other features in your mind (ie., pre-defined patterns)

In [47]:
from bs4 import BeautifulSoup
import csv
import requests
import pandas as pd
def GetGreatestMovies(page):
  sample_html = requests.get('https://www.imdb.com/list/ls050782187/?page='+str(page)).text#get url
  beauty = BeautifulSoup(sample_html,'lxml') #creating object
  titles = beauty.find_all('h3',class_='lister-item-header')
  return titles
print("file saved")
with open('/content/sample_data/new.csv','w') as csv_file:
  for page in range(1,6): 
    csvwriter = csv.DictWriter(csv_file, fieldnames=["titleID", "title"])
    csvwriter.writeheader()
    i=1
    titles = GetGreatestMovies(page)
    for title in titles:
      if(title.text!= ""):
        csvwriter.writerow({"titleID": str(i), "title": title.text.replace("\n","")})
        i=i+1
dataframe = pd.read_csv('/content/sample_data/new.csv',sep=',',engine ='python', header=0)

#remove punctuation
dataframe["punctuationremoval"] = dataframe["title"].str.replace('[^\w\s]','')

#remove special chars
dataframe['special_chars'] = dataframe["punctuationremoval"].str.replace('[^A-Za-z0-9 ]+','')
#remove Numbers
dataframe['nums_removed'] = dataframe["special_chars"].str.replace('[^A-Za-z ]+','')
#extract stopwords
import requests
data = requests.get("https://gist.githubusercontent.com/sebleier/554280/raw/7e0e4a1ce04c2bb7bd41089c9821dbcf6d0c786c/NLTK's%2520list%2520of%2520english%2520stopwords")
a = data.text
stop = a.split()
#stopwords
dataframe['stopwords'] = dataframe["nums_removed"].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

#lowercase
dataframe["lower_case"] = dataframe['stopwords'].apply(lambda x: " ".join(x.lower() for x in x.split()))

#tokenizing data for next step - stemming
import nltk
from textblob import TextBlob
nltk.download('punkt')
dataframe["tokenization"] = dataframe["lower_case"].apply(lambda x: TextBlob(x).words)
dataframe

#stemming
from nltk.stem import PorterStemmer
st = PorterStemmer()
dataframe["stemming"]= dataframe["tokenization"].apply(lambda x: " ".join([st.stem(word) for word in x]))

#Lemmatization
from textblob import Word
import nltk
nltk.download('wordnet')
dataframe["lemmatization"] = dataframe["stemming"].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
dataframe
# Parts of Speech (POS) Tagging
nltk.download('averaged_perceptron_tagger')
from nltk.tokenize import word_tokenize
sample_list =[]
for sent in dataframe["lemmatization"]:
  text = word_tokenize(sent)
  sample_list.append(nltk.pos_tag(text))
sample_list

file saved
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


[[('the', 'DT'), ('godfath', 'NN')],
 [('the', 'DT'), ('silenc', 'NN'), ('lamb', 'NN')],
 [('star', 'JJ'),
  ('war', 'NN'),
  ('episod', 'NN'),
  ('v', 'IN'),
  ('the', 'DT'),
  ('empir', 'NN'),
  ('strike', 'NN'),
  ('back', 'RB')],
 [('the', 'DT'), ('shawshank', 'NN'), ('redempt', 'NN')],
 [('the', 'DT'), ('shine', 'NN')],
 [('casablanca', 'NN')],
 [('one', 'CD'),
  ('flew', 'NN'),
  ('over', 'IN'),
  ('cuckoo', 'NN'),
  ('nest', 'JJS')],
 [('indiana', 'NN'),
  ('jone', 'NN'),
  ('raider', 'NN'),
  ('lost', 'VBD'),
  ('ark', 'NN')],
 [('the', 'DT'),
  ('lord', 'NN'),
  ('ring', 'VBG'),
  ('the', 'DT'),
  ('return', 'NN'),
  ('king', 'NN')],
 [('star', 'JJ'),
  ('war', 'NN'),
  ('episod', 'NN'),
  ('iv', 'VB'),
  ('a', 'DT'),
  ('new', 'JJ'),
  ('hope', 'NN')],
 [('the', 'DT'), ('dark', 'NN'), ('knight', 'NN')],
 [('the', 'DT'), ('godfath', 'NN'), ('part', 'NN'), ('ii', 'NN')],
 [('alien', 'NN')],
 [('schindler', 'NN'), ('list', 'NN')],
 [('incept', 'NN')],
 [('the', 'DT'),
  ('lord',

In [48]:
#tf-idf
tf1 = (dataframe["lemmatization"]).apply(lambda x: pd.value_counts(x.split(" "))).sum(axis = 0).reset_index()
tf1.columns = ['words','tf']
tf1

,words,tf
0,godfath,3.0
1,the,107.0
2,silenc,1.0
3,lamb,1.0
4,v,3.0
...,...,...
836,limit,1.0
837,darjeel,1.0
838,planet,1.0
839,terror,1.0


In [49]:
#POS-tag features
noun = []
determiner = []
adjective = []
personalpro = []
for item in sample_list:
  nouncount = 0
  detcount = 0
  adjcount = 0
  personalprocount = 0
  for word,tag in item:
    if tag == 'NN':
      nouncount = nouncount +1
    elif tag == 'DT':
      detcount = detcount +1
    elif tag == 'JJ':
      adjcount = adjcount +1
    elif tag == 'PRP':
      personalprocount = personalprocount +1
  noun.append(nouncount)
  determiner.append(detcount)
  adjective.append(adjcount)
  personalpro.append(personalprocount)
dataframe['nouncount'] = noun
dataframe['determinercount'] = determiner
dataframe['adjectivecount'] = adjective
dataframe['personalpronouncount'] = personalpro
dataframe

,titleID,title,punctuationremoval,special_chars,nums_removed,stopwords,lower_case,tokenization,stemming,lemmatization,nouncount,determinercount,adjectivecount,personalpronouncount
0,1,1.The Godfather(1972),1The Godfather1972,1The Godfather1972,The Godfather,The Godfather,the godfather,"[the, godfather]",the godfath,the godfath,1,1,0,0
1,2,2.The Silence of the Lambs(1991),2The Silence of the Lambs1991,2The Silence of the Lambs1991,The Silence of the Lambs,The Silence Lambs,the silence lambs,"[the, silence, lambs]",the silenc lamb,the silenc lamb,2,1,0,0
2,3,3.Star Wars: Episode V - The Empire Strikes Ba...,3Star Wars Episode V The Empire Strikes Back1980,3Star Wars Episode V The Empire Strikes Back1980,Star Wars Episode V The Empire Strikes Back,Star Wars Episode V The Empire Strikes Back,star wars episode v the empire strikes back,"[star, wars, episode, v, the, empire, strikes,...",star war episod v the empir strike back,star war episod v the empir strike back,4,1,1,0
3,4,4.The Shawshank Redemption(1994),4The Shawshank Redemption1994,4The Shawshank Redemption1994,The Shawshank Redemption,The Shawshank Redemption,the shawshank redemption,"[the, shawshank, redemption]",the shawshank redempt,the shawshank redempt,2,1,0,0
4,5,5.The Shining(1980),5The Shining1980,5The Shining1980,The Shining,The Shining,the shining,"[the, shining]",the shine,the shine,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499,96,"496.Me, Myself & Irene(2000)",496Me Myself Irene2000,496Me Myself Irene2000,Me Myself Irene,Me Myself Irene,me myself irene,"[me, myself, irene]",me myself iren,me myself iren,0,0,0,2
500,97,497.The Darjeeling Limited(2007),497The Darjeeling Limited2007,497The Darjeeling Limited2007,The Darjeeling Limited,The Darjeeling Limited,the darjeeling limited,"[the, darjeeling, limited]",the darjeel limit,the darjeel limit,2,1,0,0
501,98,498.Fear(1996),498Fear1996,498Fear1996,Fear,Fear,fear,[fear],fear,fear,1,0,0,0
502,99,499.Planet Terror(2007),499Planet Terror2007,499Planet Terror2007,Planet Terror,Planet Terror,planet terror,"[planet, terror]",planet terror,planet terror,2,0,0,0


In [50]:
dataframe['SentenceLength'] = dataframe['lemmatization'].apply(lambda x: len(x))
dataframe

,titleID,title,punctuationremoval,special_chars,nums_removed,stopwords,lower_case,tokenization,stemming,lemmatization,nouncount,determinercount,adjectivecount,personalpronouncount,SentenceLength
0,1,1.The Godfather(1972),1The Godfather1972,1The Godfather1972,The Godfather,The Godfather,the godfather,"[the, godfather]",the godfath,the godfath,1,1,0,0,11
1,2,2.The Silence of the Lambs(1991),2The Silence of the Lambs1991,2The Silence of the Lambs1991,The Silence of the Lambs,The Silence Lambs,the silence lambs,"[the, silence, lambs]",the silenc lamb,the silenc lamb,2,1,0,0,15
2,3,3.Star Wars: Episode V - The Empire Strikes Ba...,3Star Wars Episode V The Empire Strikes Back1980,3Star Wars Episode V The Empire Strikes Back1980,Star Wars Episode V The Empire Strikes Back,Star Wars Episode V The Empire Strikes Back,star wars episode v the empire strikes back,"[star, wars, episode, v, the, empire, strikes,...",star war episod v the empir strike back,star war episod v the empir strike back,4,1,1,0,39
3,4,4.The Shawshank Redemption(1994),4The Shawshank Redemption1994,4The Shawshank Redemption1994,The Shawshank Redemption,The Shawshank Redemption,the shawshank redemption,"[the, shawshank, redemption]",the shawshank redempt,the shawshank redempt,2,1,0,0,21
4,5,5.The Shining(1980),5The Shining1980,5The Shining1980,The Shining,The Shining,the shining,"[the, shining]",the shine,the shine,1,1,0,0,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499,96,"496.Me, Myself & Irene(2000)",496Me Myself Irene2000,496Me Myself Irene2000,Me Myself Irene,Me Myself Irene,me myself irene,"[me, myself, irene]",me myself iren,me myself iren,0,0,0,2,14
500,97,497.The Darjeeling Limited(2007),497The Darjeeling Limited2007,497The Darjeeling Limited2007,The Darjeeling Limited,The Darjeeling Limited,the darjeeling limited,"[the, darjeeling, limited]",the darjeel limit,the darjeel limit,2,1,0,0,17
501,98,498.Fear(1996),498Fear1996,498Fear1996,Fear,Fear,fear,[fear],fear,fear,1,0,0,0,4
502,99,499.Planet Terror(2007),499Planet Terror2007,499Planet Terror2007,Planet Terror,Planet Terror,planet terror,"[planet, terror]",planet terror,planet terror,2,0,0,0,13
